In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
#from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

%matplotlib inline

In [3]:
df = pd.read_csv("2018_qc_cutoff_diff_temp_vgrad.csv")
#df2019 = pd.read_csv("2019_qc_cutoff_diff_temp_vgrad.csv")
#df = pd.concat([df2018,df2019])

In [13]:
colAnaList = ['diff_temp','vgrad']
cluAnaList = [2,3,5,20,40]

for col in colAnaList:
    for limit in cluAnaList:
        df["std_{0}_{1}".format(limit,col)] = (
            df.groupby('profile')[col].rolling(window=limit,min_periods=0)
            .std().round(decimals=3).reset_index()[col])
        df["avg_{0}_{1}".format(limit,col)] = (
            df.groupby('profile')[col].rolling(window=limit,min_periods=0)
            .mean().shift(1).round(decimals=3).reset_index()[col])
        df["norm_{0}_{1}".format(limit,col)] = round(
            (np.abs(df[col]-df["avg_{0}_{1}".format(limit,col)]))
            .divide(df["std_{0}_{1}".format(limit,col)]),3)
        
        for nexnum in np.arange(-1,-4,-1):
            df["prev_{0}_std_{1}_{2}".format(abs(nexnum),limit,col)] = (
                df.groupby('profile')["std_{0}_{1}".format(limit,col)].shift(abs(nexnum)))
            df["prev_{0}_avg_{1}_{2}".format(abs(nexnum),limit,col)] = (
                df.groupby('profile')["avg_{0}_{1}".format(limit,col)].shift(abs(nexnum)))
            df["prev_{0}_norm_{1}_{2}".format(abs(nexnum),limit,col)] = (
                df.groupby('profile')["norm_{0}_{1}".format(limit,col)].shift(abs(nexnum)))

            df["next_{0}_std_{1}_{2}".format(abs(nexnum),limit,col)] = (
                df.groupby('profile')["std_{0}_{1}".format(limit,col)].shift(nexnum))
            df["next_{0}_avg_{1}_{2}".format(abs(nexnum),limit,col)] = (
                df.groupby('profile')["avg_{0}_{1}".format(limit,col)].shift(nexnum))
            df["next_{0}_norm_{1}_{2}".format(abs(nexnum),limit,col)] = (
                df.groupby('profile')["norm_{0}_{1}".format(limit,col)].shift(nexnum))



In [14]:
df2 = df[df['away_from_last_point'].between(-20,7)]

In [17]:
df2.to_csv("qc_pass_2018_2019_cutoff_analysis.csv")